### Preprocessing


In [1]:
import os
import shutil
import random

color_folders_dir = "Data/VCoR"

cleandata_dir = "Data/cleandata"

if not os.path.exists(cleandata_dir):
    os.makedirs(cleandata_dir)


# Function to select and copy images from a color folder to a target directory
def select_and_copy_images(color_folder, percentage, target_dir):
    # List all files in the color folder
    all_files = [
        f
        for f in os.listdir(color_folder)
        if os.path.isfile(os.path.join(color_folder, f))
    ]

    number_of_files_to_select = max(1, int(len(all_files) * percentage))
    selected_files = random.sample(all_files, number_of_files_to_select)

    for file_name in selected_files:
        src_file_path = os.path.join(color_folder, file_name)
        dst_file_path = os.path.join(target_dir, file_name)

        # Check if file already exists in target directory
        if os.path.exists(dst_file_path):
            # If it exists, prepend the color name to the filename to avoid overwriting
            dst_file_path = os.path.join(
                target_dir, f"{os.path.basename(color_folder)}_{file_name}"
            )

        shutil.copy2(src_file_path, dst_file_path)


# Get a list of all the color folders
color_folders = [
    os.path.join(color_folders_dir, f)
    for f in os.listdir(color_folders_dir)
    if os.path.isdir(os.path.join(color_folders_dir, f))
]

# Process each color folder
for folder in color_folders:
    select_and_copy_images(folder, 0.10, cleandata_dir)

print("Images have been selected and copied to the 'cleandata' folder.")

Images have been selected and copied to the 'cleandata' folder.


### Yolo Training


In [4]:
import ultralytics

ultralytics.checks()

Ultralytics YOLOv8.1.47  Python-3.8.19 torch-1.10.1+cu113 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
Setup complete  (16 CPUs, 63.9 GB RAM, 864.7/931.5 GB disk)


In [5]:
from ultralytics import YOLO

In [6]:
import os

data_path = "./Data/Classified_data"
print("Validating paths:")
for subfolder in ["train", "val", "test"]:
    full_path = os.path.join(data_path, subfolder)
    if os.path.exists(full_path):
        print(f"Path '{full_path}' exists.")
    else:
        print(f"Path '{full_path}' does NOT exist.")

Validating paths:
Path './Data/Classified_data\train' exists.
Path './Data/Classified_data\val' exists.
Path './Data/Classified_data\test' exists.


### Hyperparameter Tuning


In [7]:
model = YOLO("yolov8n-cls.pt")
model.tune(
    data="./Data/Classified_data",
    epochs=30,
    iterations=300,
    optimizer="AdamW",
    plots=True,
    save=True,
    val=True,
)

Tuner: Initialized Tuner instance with 'tune_dir=runs\classify\tune17'
Tuner:  Learn about tuning at https://docs.ultralytics.com/guides/hyperparameter-tuning


Tuner: Starting iteration 1/300 with hyperparameters: {'lr0': 0.01, 'lrf': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8, 'box': 7.5, 'cls': 0.5, 'dfl': 1.5, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'degrees': 0.0, 'translate': 0.1, 'scale': 0.5, 'shear': 0.0, 'perspective': 0.0, 'flipud': 0.0, 'fliplr': 0.5, 'bgr': 0.0, 'mosaic': 1.0, 'mixup': 0.0, 'copy_paste': 0.0}
Saved runs\classify\tune17\tune_scatter_plots.png
Saved runs\classify\tune17\tune_fitness.png

Tuner: 1/300 iterations complete  (102.76s)
Tuner: Results saved to runs\classify\tune17
Tuner: Best fitness=0.94643 observed at iteration 1
Tuner: Best fitness metrics are {'metrics/accuracy_top1': 0.89286, 'metrics/accuracy_top5': 1.0, 'val/loss': 0.71367, 'fitness': 0.94643}
Tuner: Best fitness model is runs\classify\train3
Tuner: Best fitness hyperparameters are printed below.

Printing 'runs\classify\tune17\best_hyperparameters.yaml'

lr0: 0.01
lrf: 0.01
momentum: 0.937
wei

KeyboardInterrupt: 

### Training


In [10]:
model = YOLO("yolov8n-cls.pt")

# Updated training configuration using the best hyperparameters from the tuning process
model.train(
    data="./Data/Classified_data",
    cfg="./runs/classify/tune17/best_hyperparameters.yaml",
)

Ultralytics YOLOv8.1.47  Python-3.8.19 torch-1.10.1+cu113 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
WARNING  Upgrade to torch>=2.0.0 for deterministic training.
engine\trainer: task=classify, mode=train, model=None, data=./Data/Classified_data, epochs=100, time=None, patience=100, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train168, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, sa

train: Scanning I:\Study\SFU\Spring2024\IAT885\Final_Project\Data\Classified_data\train... 500 images, 0 corrupt: 100%|██████████| 500/500 [00:00<?, ?it/s]
i:\Study\anaconda\envs\yolov8\lib\site-packages\torchvision\transforms\transforms.py:852: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
val: Scanning I:\Study\SFU\Spring2024\IAT885\Final_Project\Data\Classified_data\val... 140 images, 0 corrupt: 100%|██████████| 140/140 [00:00<?, ?it/s]
i:\Study\anaconda\envs\yolov8\lib\site-packages\torchvision\transforms\transforms.py:287: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.00937' and 'momentum=0.96802' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 26 weight(decay=0.0), 27 weight(decay=0.00043), 27 bias(decay=0.0)
Image sizes 224 train, 224 val
Using 8 dataloader workers
Logging results to runs\classify\train168
Starting training for 100 epochs...

      Epoch    GPU_mem       loss  Instances       Size


      1/100     0.208G     0.9509          4        224: 100%|██████████| 32/32 [00:03<00:00,  9.03it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 5/5 [00:01<00:00,  3.80it/s]

                   all      0.636          1


...

                   all      0.893          1



100 epochs completed in 0.045 hours.
Optimizer stripped from runs\classify\train168\weights\last.pt, 3.0MB
Optimizer stripped from runs\classify\train168\weights\best.pt, 3.0MB

Validating runs\classify\train168\weights\best.pt...
Ultralytics YOLOv8.1.47  Python-3.8.19 torch-1.10.1+cu113 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
YOLOv8n-cls summary (fused): 73 layers, 1438723 parameters, 0 gradients, 3.3 GFLOPs
train: I:\Study\SFU\Spring2024\IAT885\Final_Project\Data\Classified_data\train... found 500 images in 3 classes  
val: I:\Study\SFU\Spring2024\IAT885\Final_Project\Data\Classified_data\val... found 140 images in 3 classes  
test: I:\Study\SFU\Spring2024\IAT885\Final_Project\Data\Classified_data\test... found 69 images in 3 classes  


               classes   top1_acc   top5_acc: 100%|██████████| 5/5 [00:00<00:00, 46.63it/s]


                   all      0.929          1
Speed: 0.1ms preprocess, 0.5ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs\classify\train168
Results saved to runs\classify\train168


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002191A71A220>
curves: []
curves_results: []
fitness: 0.9642857313156128
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.9285714626312256, 'metrics/accuracy_top5': 1.0, 'fitness': 0.9642857313156128}
save_dir: WindowsPath('runs/classify/train168')
speed: {'preprocess': 0.10730539049421038, 'inference': 0.45797484261648996, 'loss': 0.0, 'postprocess': 0.0}
task: 'classify'
top1: 0.9285714626312256
top5: 1.0

### Evaluation Using the Val Folder


In [11]:
model = YOLO("./runs/classify/train168/weights/best.pt")

metrics = model.val()
metrics.top1
metrics.top5

Ultralytics YOLOv8.1.47  Python-3.8.19 torch-1.10.1+cu113 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
YOLOv8n-cls summary (fused): 73 layers, 1438723 parameters, 0 gradients, 3.3 GFLOPs
train: I:\Study\SFU\Spring2024\IAT885\Final_Project\Data\Classified_data\train... found 500 images in 3 classes  
val: I:\Study\SFU\Spring2024\IAT885\Final_Project\Data\Classified_data\val... found 140 images in 3 classes  
test: I:\Study\SFU\Spring2024\IAT885\Final_Project\Data\Classified_data\test... found 69 images in 3 classes  


val: Scanning I:\Study\SFU\Spring2024\IAT885\Final_Project\Data\Classified_data\val... 140 images, 0 corrupt: 100%|██████████| 140/140 [00:00<?, ?it/s]
i:\Study\anaconda\envs\yolov8\lib\site-packages\torchvision\transforms\transforms.py:287: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
               classes   top1_acc   top5_acc: 100%|██████████| 9/9 [00:04<00:00,  1.90it/s]


                   all      0.929          1
Speed: 0.1ms preprocess, 0.5ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs\classify\val


1.0

### Prediction Using Separate Testing Dataset


In [15]:
model = YOLO("./runs/classify/train168/weights/best.pt")
results = model("./predictions")


image 1/90 i:\Study\SFU\Spring2024\IAT885\Final_Project\predictions\02e37c0e56.jpg: 224x224 small 0.94, medium 0.06, large 0.00, 3.6ms
image 2/90 i:\Study\SFU\Spring2024\IAT885\Final_Project\predictions\05aeb6ddec.jpg: 224x224 small 0.99, medium 0.01, large 0.00, 7.6ms
image 3/90 i:\Study\SFU\Spring2024\IAT885\Final_Project\predictions\0b01008bee.jpg: 224x224 small 1.00, medium 0.00, large 0.00, 6.5ms
image 4/90 i:\Study\SFU\Spring2024\IAT885\Final_Project\predictions\0c3bb456ee.jpg: 224x224 small 0.58, medium 0.42, large 0.00, 4.0ms
image 5/90 i:\Study\SFU\Spring2024\IAT885\Final_Project\predictions\0e491569c8.jpg: 224x224 small 1.00, medium 0.00, large 0.00, 4.0ms
image 6/90 i:\Study\SFU\Spring2024\IAT885\Final_Project\predictions\148b514638.jpg: 224x224 small 1.00, medium 0.00, large 0.00, 8.5ms
image 7/90 i:\Study\SFU\Spring2024\IAT885\Final_Project\predictions\1a1296c730.jpg: 224x224 small 1.00, medium 0.00, large 0.00, 4.5ms
image 8/90 i:\Study\SFU\Spring2024\IAT885\Final_Projec

i:\Study\anaconda\envs\yolov8\lib\site-packages\torchvision\transforms\transforms.py:287: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


image 15/90 i:\Study\SFU\Spring2024\IAT885\Final_Project\predictions\2e18679b94.jpg: 224x224 small 0.83, medium 0.17, large 0.00, 5.5ms
image 16/90 i:\Study\SFU\Spring2024\IAT885\Final_Project\predictions\33fe711bdb.jpg: 224x224 small 1.00, medium 0.00, large 0.00, 4.5ms
image 17/90 i:\Study\SFU\Spring2024\IAT885\Final_Project\predictions\3451392eb0.jpg: 224x224 small 0.88, medium 0.11, large 0.00, 4.0ms
image 18/90 i:\Study\SFU\Spring2024\IAT885\Final_Project\predictions\408174870e.jpg: 224x224 small 1.00, medium 0.00, large 0.00, 7.5ms
image 19/90 i:\Study\SFU\Spring2024\IAT885\Final_Project\predictions\423683c20d.jpg: 224x224 small 1.00, large 0.00, medium 0.00, 6.5ms
image 20/90 i:\Study\SFU\Spring2024\IAT885\Final_Project\predictions\429b8a9ea7.jpg: 224x224 large 1.00, medium 0.00, small 0.00, 5.0ms
image 21/90 i:\Study\SFU\Spring2024\IAT885\Final_Project\predictions\431dd13304.jpg: 224x224 small 0.96, medium 0.04, large 0.00, 7.0ms
image 22/90 i:\Study\SFU\Spring2024\IAT885\Final